<a href="https://colab.research.google.com/github/naoto-git/for_submit/blob/main/101_knock_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 128kB/s 
     |████████████████████████████████| 983kB 35.6MB/s 
     |████████████████████████████████| 245kB 40.3MB/s 
     |████████████████████████████████| 1.3MB 44.9MB/s 
     |████████████████████████████████| 471kB 40.2MB/s 
     |████████████████████████████████| 1.1MB 36.7MB/s 
     |████████████████████████████████| 890kB 37.7MB/s 
     |████████████████████████████████| 2.9MB 37.3MB/s 
  Created wheel for ktrain: filename=ktrain-0.23.0-cp36-none-any.whl size=25271671 sha256=86146dbf390ba142594f225ddb1fea1bfe2bb943fa37e66c38f7bb6780c7b4b4
  Stored in directory: /root/.cache/pip/wheels/f2/15/d1/ada9c041daebeb09b7341e2d3d0a765bf74e512edf2caa6fa7
  Created wheel for keras-bert: filename=keras_bert-0.86.0-cp36-none-any.whl size=34145 sha256=f56bdd45308699e97e83aa815c93d4188d2be18ef7ed8b745e6338d6ad6dd52e
  Stored in directory: /root/.cache/pip/wheels/66/f0/b1/748128b58562fc9e31b907bb5e2ab6a35eb37695e83911236b
  Created wheel for lan

In [15]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [16]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12413520010497929404, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 6221232419836506020
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 17712718372331308011
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15695549568
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 1903645533948618087
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

In [17]:
from sklearn.datasets import fetch_20newsgroups
categories = ["alt.atheism", "soc.religion.christian", "comp.graphics", "sci.med"]
train_b = fetch_20newsgroups(subset="train", categories=categories, shuffle=True, random_state=42)
test_b = fetch_20newsgroups(subset="test", categories=categories, shuffle=True, random_state=42)
print("size of training set: %s" %(len(train_b["data"])))
print("size of classes set: %s" %(len(train_b.target_names)))

size of training set: 2257
size of classes set: 4


In [47]:
train_b['target_names']

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [18]:
x_train = train_b.data
x_test = test_b.data
y_train = train_b.target
y_test = test_b.target

In [19]:
print(train_b.data[0], train_b.target[0])

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton
Organization: The City University
Lines: 14

Does anyone know of a good way (standard PC application/PD utility) to
convert tif/img/tga files into LaserJet III format.  We would also like to
do the same, converting to HPGL (HP plotter) files.

Please email any response.

Is this the correct group?

Thanks in advance.  Michael.
-- 
Michael Collier (Programmer)                 The Computer Unit,
Email: M.P.Collier@uk.ac.city                The City University,
Tel: 071 477-8000 x3769                      London,
Fax: 071 477-8565                            EC1V 0HB.
 1


In [20]:
classes = ["alt.atheism", "comp.graphics", "sci.med", "soc.religion.christian"]
classes[train_b.target[0]]

'comp.graphics'

In [21]:
import ktrain
from ktrain import text

(x_train, y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test, 
                                                  class_names = train_b.target_names, preprocess_mode="bert", maxlen=250, max_features=35000)

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


In [25]:
model = text.text_classifier("bert", train_data=(x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(model, train_data=(x_train, y_train), batch_size=6)

Is Multi-Label? False
maxlen is 250
done.


In [26]:
learner.fit_onecycle(2e-5, 4)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/4
377/377 [==============================] - 169s 449ms/step - loss: 0.6435 - accuracy: 0.7470
Epoch 2/4
377/377 [==============================] - 169s 449ms/step - loss: 0.1344 - accuracy: 0.9566
Epoch 3/4
377/377 [==============================] - 169s 450ms/step - loss: 0.0456 - accuracy: 0.9889
Epoch 4/4
377/377 [==============================] - 169s 449ms/step - loss: 0.0089 - accuracy: 0.9987


In [27]:
learner.validate(val_data=(x_test, y_test), class_names=train_b.target_names)

                        precision    recall  f1-score   support

           alt.atheism       0.96      0.92      0.94       319
         comp.graphics       0.97      0.98      0.97       389
               sci.med       0.98      0.96      0.97       396
soc.religion.christian       0.94      0.98      0.96       398

              accuracy                           0.96      1502
             macro avg       0.96      0.96      0.96      1502
          weighted avg       0.96      0.96      0.96      1502



array([[293,   6,   4,  16],
       [  4, 381,   2,   2],
       [  4,   4, 382,   6],
       [  5,   2,   2, 389]])

In [30]:
from sklearn.datasets import fetch_20newsgroups

categories = ["alt.atheism", "comp.graphics", "sci.med", "soc.religion.christian"]
train_b = fetch_20newsgroups(subset="train", categories=categories, shuffle=True)
test_b = fetch_20newsgroups(subset="test", categories=categories, shuffle=True)
(x_train, y_train) = (train_b.data, train_b.target)
(x_test, y_test) = (test_b.data, test_b.target)

import ktrain
from ktrain import text
MODEL_NAME = "distilbert-base-uncased"

t = text.Transformer(MODEL_NAME, maxlen=250, class_names=train_b.target_names)
trn = t.preprocess_train(x_train, y_train)
val = t.preprocess_test(x_test, y_test)
model = t.get_classifier()


preprocessing train...
language: en
train sequence lengths:
	mean : 308
	95percentile : 837
	99percentile : 1938


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 343
	95percentile : 979
	99percentile : 2562


In [31]:
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)
learner.fit_onecycle(5e-5, 4)



begin training using onecycle policy with max lr of 5e-05...
Epoch 1/4
377/377 [==============================] - 54s 144ms/step - loss: 0.5744 - accuracy: 0.8095 - val_loss: 0.2462 - val_accuracy: 0.9214
Epoch 2/4
377/377 [==============================] - 53s 141ms/step - loss: 0.1342 - accuracy: 0.9597 - val_loss: 0.1955 - val_accuracy: 0.9427
Epoch 3/4
377/377 [==============================] - 53s 140ms/step - loss: 0.0562 - accuracy: 0.9832 - val_loss: 0.1484 - val_accuracy: 0.9574
Epoch 4/4
377/377 [==============================] - 53s 140ms/step - loss: 0.0135 - accuracy: 0.9965 - val_loss: 0.1652 - val_accuracy: 0.9581


In [38]:
learner.validate(val_data=val, class_names=t.get_classes())

                        precision    recall  f1-score   support

           alt.atheism       0.92      0.95      0.93       319
         comp.graphics       0.97      0.95      0.96       389
               sci.med       0.96      0.97      0.97       396
soc.religion.christian       0.98      0.96      0.97       398

              accuracy                           0.96      1502
             macro avg       0.96      0.96      0.96      1502
          weighted avg       0.96      0.96      0.96      1502



array([[302,   4,   8,   5],
       [ 13, 371,   4,   1],
       [  4,   6, 385,   1],
       [ 10,   3,   4, 381]])

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

{'batch_size': None,
 'c': ['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian'],
 'config': DistilBertConfig {
   "_name_or_path": "distilbert-base-uncased",
   "activation": "gelu",
   "architectures": [
     "DistilBertForMaskedLM"
   ],
   "attention_dropout": 0.1,
   "dim": 768,
   "dropout": 0.1,
   "hidden_dim": 3072,
   "id2label": {
     "0": "LABEL_0",
     "1": "LABEL_1",
     "2": "LABEL_2",
     "3": "LABEL_3"
   },
   "initializer_range": 0.02,
   "label2id": {
     "LABEL_0": 0,
     "LABEL_1": 1,
     "LABEL_2": 2,
     "LABEL_3": 3
   },
   "max_position_embeddings": 512,
   "model_type": "distilbert",
   "n_heads": 12,
   "n_layers": 6,
   "pad_token_id": 0,
   "qa_dropout": 0.1,
   "seq_classif_dropout": 0.2,
   "sinusoidal_pos_embds": false,
   "tie_weights_": true,
   "vocab_size": 30522
 },
 'lang': 'en',
 'max_features': 10000,
 'maxlen': 250,
 'model_name': 'distilbert-base-uncased',
 'model_type': transformers.modeling_tf_auto.TFAutoModelForSequ